# Advanced Time Series Forecasting for Retail: A Comparative Study

# Author: Amina Abacon

# Part 4: Neural Networks (No Exogenous Variables)

In [1]:
# Import data and packages

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS, NBEATS, PatchTST
from neuralforecast.losses.pytorch import MAE
import torch

print("=" * 70)
print("PART 4: NEURAL NETWORKS (NO EXOGENOUS VARIABLES)")
print("=" * 70)

# ============================================================================
# STEP 1: Load Data
# ============================================================================
print("\n[1] Loading Data...")
print("-" * 70)

df = pd.read_csv('data/processed/train_with_features.csv', parse_dates=['ds'])

print(f"✓ Loaded data: {df.shape}")
print(f"Time Series: {df['unique_id'].nunique()}")
print(f"Date Range: {df['ds'].min().date()} to {df['ds'].max().date()}")

PART 4: NEURAL NETWORKS (NO EXOGENOUS VARIABLES)

[1] Loading Data...
----------------------------------------------------------------------
✓ Loaded data: (109650, 17)
Time Series: 150
Date Range: 2015-08-16 to 2017-08-15


In [2]:
# ============================================================================
# STEP 2: Train/Test Split
# ============================================================================
print("\n[2] Creating Train/Test Split...")
print("-" * 70)

test_days = 7
max_date = df['ds'].max()
test_start = max_date - pd.Timedelta(days=test_days - 1)

train_df = df[df['ds'] < test_start].copy()
test_df = df[df['ds'] >= test_start].copy()

print(f"Train Set: {train_df['ds'].min().date()} to {train_df['ds'].max().date()}")
print(f"  Records: {len(train_df):,}")

print(f"\nTest Set: {test_df['ds'].min().date()} to {test_df['ds'].max().date()}")
print(f"  Records: {len(test_df):,}")

# ============================================================================
# STEP 3: Prepare Data for NeuralForecast
# ============================================================================
print("\n[3] Preparing Data for NeuralForecast...")
print("-" * 70)

# NeuralForecast needs: unique_id, ds, y (no exogenous for this part)
neural_train = train_df[['unique_id', 'ds', 'y']].copy()
neural_train = neural_train.sort_values(['unique_id', 'ds']).reset_index(drop=True)

print(f"✓ Training data prepared: {neural_train.shape}")
print(f"✓ Time series: {neural_train['unique_id'].nunique()}")
print(f"✓ Average series length: {len(neural_train) / neural_train['unique_id'].nunique():.0f} days")


[2] Creating Train/Test Split...
----------------------------------------------------------------------
Train Set: 2015-08-16 to 2017-08-08
  Records: 108,600

Test Set: 2017-08-09 to 2017-08-15
  Records: 1,050

[3] Preparing Data for NeuralForecast...
----------------------------------------------------------------------
✓ Training data prepared: (108600, 3)
✓ Time series: 150
✓ Average series length: 724 days


In [3]:
# ============================================================================
# STEP 4: Initialize Neural Network Models
# ============================================================================
print("\n[4] Initializing Neural Network Models...")
print("-" * 70)

forecast_horizon = 7
input_size = 7 * 4  # 4 weeks of history

print(f"\nModel Configuration:")
print(f"  Forecast Horizon: {forecast_horizon} days")
print(f"  Input Size: {input_size} days (4 weeks)")
print(f"  Loss Function: MAE")
print(f"  Training Steps: 500")
print(f"  Early Stopping: Enabled (patience=5)")

# Initialize three neural network architectures
nhits = NHITS(
    h=forecast_horizon,
    input_size=input_size,
    max_steps=500,
    early_stop_patience_steps=5,
    loss=MAE(),
    scaler_type='robust',
    random_seed=42,
)

nbeats = NBEATS(
    h=forecast_horizon,
    input_size=input_size,
    max_steps=500,
    early_stop_patience_steps=5,
    loss=MAE(),
    scaler_type='robust',
    random_seed=42,
)

patchtst = PatchTST(
    h=forecast_horizon,
    input_size=input_size,
    max_steps=500,
    early_stop_patience_steps=5,
    scaler_type='robust',
    random_seed=42,
)

models = [nhits, nbeats, patchtst]

print("\n✓ Models initialized:")
print("  1. NHITS - Neural Hierarchical Interpolation (SOTA)")
print("  2. NBEATS - Neural Basis Expansion (interpretable)")
print("  3. PatchTST - Transformer-based (cutting-edge)")

Seed set to 42



[4] Initializing Neural Network Models...
----------------------------------------------------------------------

Model Configuration:
  Forecast Horizon: 7 days
  Input Size: 28 days (4 weeks)
  Loss Function: MAE
  Training Steps: 500
  Early Stopping: Enabled (patience=5)


Seed set to 42
Seed set to 42



✓ Models initialized:
  1. NHITS - Neural Hierarchical Interpolation (SOTA)
  2. NBEATS - Neural Basis Expansion (interpretable)
  3. PatchTST - Transformer-based (cutting-edge)


In [4]:
# ============================================================================
# STEP 5: Train Models
# ============================================================================
print("\n[5] Training Neural Network Models...")
print("-" * 70)
print(" Training 3 models on 150 time series...")

nf = NeuralForecast(models=models, freq='D')

# Train with validation set for early stopping
nf.fit(df=neural_train, val_size=7)

print("\n✓ Training complete!")


[5] Training Neural Network Models...
----------------------------------------------------------------------
 Training 3 models on 150 time series...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.771     Total estimated model params size (MB)
34        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=500` reached.



✓ Training complete!


In [5]:
# ============================================================================
# STEP 6: Generate Forecasts
# ============================================================================
print("\n[6] Generating Forecasts...")
print("-" * 70)

forecasts = nf.predict()

print(f"✓ Generated forecasts: {forecasts.shape}")
print(f"\nForecast columns: {forecasts.columns.tolist()}")
print(f"\nSample forecasts:")
print(forecasts.head(10))

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.



[6] Generating Forecasts...
----------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

✓ Generated forecasts: (1050, 5)

Forecast columns: ['unique_id', 'ds', 'NHITS', 'NBEATS', 'PatchTST']

Sample forecasts:
               unique_id         ds        NHITS       NBEATS     PatchTST
0     store_10_BEVERAGES 2017-08-09  1257.434814  1216.687378  1216.624512
1     store_10_BEVERAGES 2017-08-10  1317.754761  1314.281982  1322.969238
2     store_10_BEVERAGES 2017-08-11  1399.203125  1405.639893  1412.879639
3     store_10_BEVERAGES 2017-08-12  1623.655273  1588.416992  1551.081421
4     store_10_BEVERAGES 2017-08-13  1591.757080  1526.232910  1535.395142
5     store_10_BEVERAGES 2017-08-14  1381.227905  1275.457153  1337.587402
6     store_10_BEVERAGES 2017-08-15  1510.223511  1392.361938  1489.016846
7  store_10_BREAD/BAKERY 2017-08-09   106.862144   106.374527   109.562126
8  store_10_BREAD/BAKERY 2017-08-10    99.230827    96.138901   101.607796
9  store_10_BREAD/BAKERY 2017-08-11    85.934875    82.451080    90.954803


In [6]:
# ============================================================================
# STEP 7: Evaluate Performance
# ============================================================================
print("\n[7] Evaluating Model Performance...")
print("-" * 70)

# Merge forecasts with actuals
forecasts_df = forecasts.reset_index()

eval_df = test_df[['unique_id', 'ds', 'y']].merge(
    forecasts_df,
    on=['unique_id', 'ds'],
    how='inner'
)

print(f"✓ Merged evaluation data: {eval_df.shape}")

def calculate_metrics(actual, predicted):
    """Calculate forecasting error metrics"""
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    mask = ~(np.isnan(actual) | np.isnan(predicted))
    actual = actual[mask]
    predicted = predicted[mask]
    
    if len(actual) == 0:
        return {'MAE': np.nan, 'RMSE': np.nan, 'MAPE': np.nan}
    
    mae = np.mean(np.abs(actual - predicted))
    rmse = np.sqrt(np.mean((actual - predicted) ** 2))
    mape = np.mean(np.abs((actual - predicted) / (actual + 1e-10))) * 100
    
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

# Calculate metrics for each model
model_names = ['NHITS', 'NBEATS', 'PatchTST']
results = []

for model in model_names:
    if model in eval_df.columns:
        metrics = calculate_metrics(eval_df['y'], eval_df[model])
        metrics['Model'] = model
        metrics['Type'] = 'Neural Network'
        results.append(metrics)

results_df = pd.DataFrame(results)
results_df = results_df[['Model', 'Type', 'MAE', 'RMSE', 'MAPE']].sort_values('MAE')

print("\n Neural Network Performance:")
print(results_df.to_string(index=False))

best_neural = results_df.iloc[0]
print(f"\n Best Neural Network: {best_neural['Model']}")
print(f"   MAE: ${best_neural['MAE']:.2f}")
print(f"   RMSE: ${best_neural['RMSE']:.2f}")
print(f"   MAPE: {best_neural['MAPE']:.2f}%")


[7] Evaluating Model Performance...
----------------------------------------------------------------------
✓ Merged evaluation data: (1050, 7)

 Neural Network Performance:
   Model           Type        MAE       RMSE      MAPE
PatchTST Neural Network 213.834889 465.574524 16.971101
  NBEATS Neural Network 217.471313 465.096432 16.894965
   NHITS Neural Network 225.062459 476.347403 17.365082

 Best Neural Network: PatchTST
   MAE: $213.83
   RMSE: $465.57
   MAPE: 16.97%


In [7]:
#============================================================================
# STEP 8: Compare with Statistical Baseline
# ============================================================================
print("\n[8] Comparing with Statistical Baseline...")
print("-" * 70)

# Load baseline results
baseline_results = pd.read_csv('output/baseline_results.csv')
best_statistical = baseline_results.iloc[0]

print(f"\nBest Statistical Model: {best_statistical['Model']}")
print(f"  MAE: ${best_statistical['MAE']:.2f}")

print(f"\nBest Neural Network: {best_neural['Model']}")
print(f"  MAE: ${best_neural['MAE']:.2f}")

improvement = ((best_statistical['MAE'] - best_neural['MAE']) / best_statistical['MAE']) * 100

print(f"\n{'🎉' if improvement > 0 else '⚠️'} Neural Networks vs Statistical:")
print(f"  Improvement: {improvement:+.1f}%")

if improvement > 10:
    print("  Neural networks show significant improvement!")
elif improvement > 0:
    print("  ✓ Neural networks show modest improvement")
else:
    print("   Statistical models remain competitive")


[8] Comparing with Statistical Baseline...
----------------------------------------------------------------------

Best Statistical Model: AutoTheta
  MAE: $276.44

Best Neural Network: PatchTST
  MAE: $213.83

🎉 Neural Networks vs Statistical:
  Improvement: +22.6%
  Neural networks show significant improvement!


In [8]:
# ============================================================================
# STEP 9: Save Results (Checkpoint)
# ============================================================================
print("\n[9] Saving Results...")
print("-" * 70)

output_path = Path('output')

# Save evaluation data
eval_df.to_csv(output_path / 'neural_eval_checkpoint.csv', index=False)
print(f"✓ Saved: {output_path / 'neural_eval_checkpoint.csv'}")

# Save results
results_df.to_csv(output_path / 'neural_results.csv', index=False)
print(f"✓ Saved: {output_path / 'neural_results.csv'}")

# Save forecasts
forecasts.to_csv(output_path / 'neural_forecasts.csv')
print(f"✓ Saved: {output_path / 'neural_forecasts.csv'}")

print("\n  RECOMMENDED: Restart kernel now to free memory")
print("   Then run Part 4B (analysis notebook)")


[9] Saving Results...
----------------------------------------------------------------------
✓ Saved: output\neural_eval_checkpoint.csv
✓ Saved: output\neural_results.csv
✓ Saved: output\neural_forecasts.csv

  RECOMMENDED: Restart kernel now to free memory
   Then run Part 4B (analysis notebook)


In [9]:
# ============================================================================
# Summary
# ============================================================================
print("\n" + "=" * 70)
print("NEURAL NETWORK TRAINING COMPLETE")
print("=" * 70)

print(f"\n Results Summary:")
print(f"  Models Trained: {len(model_names)}")
print(f"  Best Model: {best_neural['Model']}")
print(f"  MAE: ${best_neural['MAE']:.2f}")
print(f"  Improvement vs Baseline: {improvement:+.1f}%")

print(f"\n Checkpoint Files Saved:")
print(f"  - neural_eval_checkpoint.csv")
print(f"  - neural_results.csv")
print(f"  - neural_forecasts.csv")

print(f"\n Next Steps:")
print(f"  1. Restart kernel (Kernel → Restart)")
print(f"  2. Run Part 4B - Neural Network Analysis")
print(f"  3. Create visualizations and detailed comparison")

print("\n" + "=" * 70)


NEURAL NETWORK TRAINING COMPLETE

 Results Summary:
  Models Trained: 3
  Best Model: PatchTST
  MAE: $213.83
  Improvement vs Baseline: +22.6%

 Checkpoint Files Saved:
  - neural_eval_checkpoint.csv
  - neural_results.csv
  - neural_forecasts.csv

 Next Steps:
  1. Restart kernel (Kernel → Restart)
  2. Run Part 4B - Neural Network Analysis
  3. Create visualizations and detailed comparison



---